# Practical Assignment Statement
# BOSTANGIU LUCA-NICOLAE
## Comments are provided throughout the code. A thorough analysis of the different algorithms, heuristics, and parameter explorations can be found at the end of the notebook.

## 1. Practical Assignment Objectives
The development of this practical assignment aims for students to analyze, design, and implement solutions to a problem using the state-space search and evolutionary computation techniques taught in the Artificial Intelligence (AI) course. To this end, students will develop a programming project in Python using the Google Colab programming environment and Python notebooks.

## 2. Case Study
We will work with a classical problem, the graph coloring problem, but with a fundamental variation. Given a series of vertices $V$ and edges $A$ forming a connected undirected graph $G$, the goal is to find the minimum number of colors such that all neighbors of any given vertex have a different color from that vertex.

Take as an example a four-vertex graph like the following:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Graph_with_all_three-colourings_2.svg/2560px-Graph_with_all_three-colourings_2.svg.png" alt="drawing" width=250>

These are the combinations of solutions that can be found for the problem, but they always have one thing in common: the minimum number of colors is 3 and only 3.

The problem is given by a graph that we know is connected and undirected. We must color each vertex. The previous graph can be represented with the following adjacency matrix:


```python
problem_exmple = [[0, 1, 1, 0],
                  [1, 0, 1, 0],
                  [1, 1, 0, 1],
                  [0, 0, 1, 0],]
```

For this graph the minimum number of colors required is 3.

## 3. Development
The development of this practical assignment involves completing this python notebook to solve the problem for several different configurations using Search techniques and Genetic Algorithms. Furthermore, using this python notebook, you must show the results of the algorithm executions to draw conclusions about the problem configurations.

It is possible that the problem configuration is too large for some of the algorithms. As a general rule, if the algorithm takes more than 5 minutes to complete its execution, it can be declared that the algorithm has not found a solution in a reasonable time (and we indicate this in the results analysis).

5 different problem configurations are provided, organized in dictionaries like the example.

It is desired to apply A* and a genetic algorithm. In the genetic algorithm section, there are specific instructions for building said algorithm.

For each configuration and algorithm, a table with these characteristics should be studied (a markdown tables generator https://www.tablesgenerator.com/markdown_tables or pandas https://pandas.pydata.org/docs/user_guide/index.html can be used):

| Problem | Algorithm | Time | Benefit |
|----------|-----------|--------|-----------|
| P1       |  A*       | 3s     | 2         |
| P2       |  Gen      | 5s     | 3         |
| P3       |  A*       | 10s    | 7         |

(**ATTENTION** the values in this table have been arbitrarily decided, they do not reflect the result of the problem.)

To complete the practical assignment, it is strictly necessary to complete this last step, there will be questions in the exam related to these results.

### Checklist
This is a list of the tasks necessary to complete the practical assignment:

- Search
  - Design the problem representation
  - Design the state space expansion
  - Design an adequate cost function for the problem
  - Design an adequate heuristic for the problem (and the cost)
  - Code a generic search algorithm
- Genetic Algorithm
  - Code a chromosome
  - Code roulette wheel selection
  - Code one-point crossover
  - Code uniform mutation
  - Code a generic genetic algorithm
- General
  - Test all combinations of Problems and Algorithms
  - Represent the execution time and the benefit obtained for each algorithm-problem pair.
  - It is **essential** to explore other parameters.

### Practical advice and hints about the practical assignment
- It is recommended to develop this notebook in an IDE to improve the development experience. Everything can be developed in Colab, but it is much more comfortable locally.
- An informed heuristic is capable of offering good solutions in reasonable time even in problem 5, even if it does not find the optimal value.
- During the evaluation, it will be taken into account that variations in parameters have been explored, some heuristics have been explored, what happens when there is no heuristic, etc. It is important to explore the problem in its entirety, not just develop the sections. These explorations must also be reported in this notebook.
- Pay attention to each section, what is asked, and the incorporated advice.
- Experiment freely with the parameters of each problem. For example, what happens if the heuristic is $h(x)=0$?

## 4. Practical Assignment Regulations
For the development of the programming project, this notebook is provided as a programming project. Several problem configurations have been proposed for use in the different tests. It is allowed to create all the necessary additional functions as long as the general structure of this notebook is kept. This notebook is the only deliverable, therefore developing code outside of it is not recommended.

In addition to explaining the decisions made, it will be necessary to make a comparison of results in one or more tables, as well as include a final comparison.

The practical assignment must be carried out taking into account the following regulations:
* It is NOT allowed to alter the names, parameters, or return type of any of the provided methods.
* The use of external libraries except numpy and pandas is not allowed.
* The practical assignment will be carried out individually.
* Plagiarism of the practical assignment is strictly prohibited. The detection of plagiarism will result in a grade of 0 in the course's exam for all students involved, as well as the possibility of opening a disciplinary academic file.
* To be evaluated for the practical assignment, it is mandatory to submit it on time, having correctly implemented at least one of the requested functionalities. A late submission will be evaluated as 0.
* To be evaluated for the practical assignment, it is mandatory to have submitted it on time prior to the multiple-choice exam. The practical assignment will be evaluated in its entirety through a multiple-choice test.
* Use this notebook as a report.
* For submission, it is strictly necessary to submit the fully executed notebook. A submission that has not been successfully executed up to the last cell will be evaluated as 0. (Submit the .ipynb file)

# Search
Use the following cells to develop all the code related to search. Remember to respect this general structure and add cells always within each section.

All the code skeleton must be respected to execute the prractical assignment correctly. Otherwise, it is possible to make a mistake and end up with the wrong conclusions.

## State Management

### Initial State
We are looking for a function that, given a specific problem (represented in `dataset`), returns everything necessary to represent the problem. "Everything necessary" is the key part, you need a data structure that allows storing the necessary information.

#### Tips:
- If you need to include several groups of data, or data of different types, it is advisable to use a dictionary.
- States should never be modified once they have been created, consider using tuples whenever possible.

In [5]:
import numpy as np
import pandas as pd

problem_example = (
                  (0, 1, 1, 0, 1),
                  (1, 0, 1, 0, 0),
                  (1, 1, 0, 1, 1),
                  (0, 0, 1, 0, 0),
                  (1, 0, 1, 0, 0)
                  )


def initial_state(dataset):
    colors = [-1] * len(dataset) # i initialize the colors list with -1 ( no color ) for every node
    state = {
            "graph":dataset,
            "colors":colors,
            "nextnode":0, # we start from the first node
            }
    return state

def copy(state):
    newstate = {
                "graph":state["graph"], # this doesnt ever change
                "colors":list(state["colors"]), # copy the list without it being linked
                "nextnode":state["nextnode"],
                }
    return newstate


### State Expansion
We will develop a function that, given a state `state`, expands all the legal states of the problem. "Legal" means that we should prevent invalid states in this step, saving computation and simplifying the solution function.

#### Tips:
- This state tree can be modeled as a decision-making process. I can choose a color and paint any vertex; or vice versa, choose any vertex and assign it a color. There are several correct approaches using this method!

In [6]:
def checknode(state, currnode, currnode_color):
    # get all the adiacent nodes for the currnode
    adiacent_nodes = [i for i in range(len(state["graph"][currnode])) if state["graph"][currnode][i] == 1]
    for i in adiacent_nodes: # for all the adiacent nodes check if the currnode color is equal
        if (currnode_color == state["colors"][i]):
            return 0
    return 1

def expand(state):
    states = [] # Create an empty list of states
    currnode = state["nextnode"]
    for i in range(len(set(state["colors"]))): # for all possible colors ( minimum number of colors + 1 )
        # we make it a set and it will remove duplicates ( +1 because we dont remove -1 from the set )
        # for example, when we don`t have any colored nodes, it would be a max_color_usable of exactly 1 ( minimum number of colors + 1 )
        # also, there will never be gaps as we generate the used colors in a consecutive order
        if checknode(state, currnode, i): # returns 1 if u can color with that
            state_copy = copy(state)
            state_copy["colors"][currnode] = i # color the node
            state_copy["nextnode"] += 1 # move to the next node
            states.append(state_copy) # append the created and updated copy
    return states # Return a list of states

### Solution Reached
The solution function is the stopping criterion. It describes what a solution to a given problem is. Remember: As soon as we detect a solution, the generic search algorithm ALWAYS stops; it means that for optimal search (with costs), it will stop regardless of whether it has found the best solution or not.

For example, the Branch and Bound algorithm guarantees finding the optimal solution, but A* does not.

#### Tips:
- Think carefully about when the algorithm offers a solution. Keep in mind that the tree (if well-designed) never has invalid states.

In [7]:
def is_solution(state):
    # Returns True if the state is a solution, False otherwise
    if (state["nextnode"] == len(state["colors"])): # if the nextnode is the lastnode + 1 it means the graph is completely colored
        return True
    else: 
        return False

## Metrics

### Cost Function
Given a complete path, the cost is computed. `path`, unlike `state`, will necessarily be a list. The path is a list of states. We want to design a function $c(P)$ that faithfully represents the objective of the problem. Cost is something exact and objective, based on the observation of decisions we have already made.

#### Tip:
- Cost is one of the most sensitive parts of the algorithm. A bad cost will make it impossible to solve.
- Think about what the problem is trying to do. What `expand` movements cause the solution to worsen?

In [8]:
def cost(path): # path must contain SEVERAL states
    # Computes the cost of a path
    # This should be possible: state = path[-1]
    # Computes the cost of a complete path
    state = path[-1] # get the last state
    color_vector = [i for i in state["colors"] if i != -1] # get the color vector without -1 colors
    return len(set(color_vector))

### Heuristic(s)
The heuristic measures the remaining cost to reach the goal state from the current state. If we have a cost function $c(P)$ for the path $P = \{S_0, S_1 .. S_{N-1}\}$, we want to find a heuristic function $h(S_{N-1})$ that **approximately** estimates what remains to be added to the cost after taking the optimal decisions up to the goal state.

More specifically, consider $c(P^*)$ the optimal path starting from path $P$. Regardless of whether $P$ is optimal up to that point, $P^*$ has a series of additional steps that are optimal. Then, an optimal heuristic $h^*(S_{N-1})$ measures the following expression:

$$c(P^*) = h^*(S_{N-1}) + c(P)$$

Since the optimal heuristic is not known, we have to work with a heuristic that helps estimate as much as possible this ideal $h^*$ function. The closer $h$ is to $h^*$, the more informed the heuristic will be. But be careful, because providing information with $h$ can be much more computationally expensive than a less informed heuristic (e.g., $h(S_{N-1}) = 0$). If we do a lot of computations to find $h(S_{N-1})$, then the heuristic will be slower. We need to find a balance between how informed and how fast our heuristic is.

Furthermore, we will consider a heuristic admissible when, in estimating the optimal cost $c(P^*)$, it is an upper bound of $h(S_{N-1}) + c(P)$. This follows this expression:

$$c(P^*) \geq h(S_{N-1}) + c(P)$$

#### Tips:
- It is not necessary to find an admissible heuristic, but it is require to justify if the proposed heuristic is admissible or not.
- An informed heuristic can significantly speed up execution in larger problems.
- Remember that the heuristic and cost are related. In this problem, you have to think: How many new colors will I have to add in the future?

In [9]:
def heuristic(state):
    # Computes a heuristic for a state
    used_colors = set(i for i in state["colors"] if i != -1) # as explained before, it also contains -1 and we now want to not have it
    heuristic = 0

    for currnode in range(state["nextnode"], len(state["colors"])): # uncolored nodes
        neighbour_colors = set()
        for i in range(len(state["graph"][currnode])): # already colored neighbouring nodes
            if state["graph"][currnode][i] == 1 and state["colors"][i] != -1:
                neighbour_colors.add(state["colors"][i])
        if neighbour_colors == used_colors and len(used_colors) > 0: # all existing colors are not applicable
            heuristic  = heuristic + 1 # so we need a new color
    # Computes the heuristic of a state
    # How much cost do we have to contribute to reach the goal state?
    # I tested this first with h=0, which was too slow and uninformed
    # Then, i used another simple heuristic = len(state["colors"]) - state["nextnode"] to return the remaining uncolored nodes
    # But it was not so well informed, so i switch to the active one, which is not addmisibile but well informed with good speed
    # it can overestimate because uncolored nodes that are not adjacent to each other might be colored with the same new color
    # and this heuristic would count them separately. So A* with this h may not be optimal, but very fast nevertheless.
    # This being said, I used both informed and uninformed heuristics, testing the outcomes, but chose the informed in the end.
    return heuristic

## Search Algorithm

### Pruning
Pruning compares two paths, observing the final states of each. `path[-1]` is the state we want to observe. After comparing the equality between two states, we have to compare $c(P_a)$ with $c(P_b)$. We only keep the one with the lower cost. If they are equal, choose any one.

#### Tip:
- The `eq` function exists to compare two states, use it to compare the last states in the pruning.
- `prune` receives a sorted list, remember this when removing duplicate states.

In [10]:
def eq(s1, s2):
    return s1["colors"] == s2["colors"] # return true if the states are equal; false otherwise

def prune(path_list):
    # If it detects that two paths lead to the same state,
    # we are only interested in the path with the lowest cost.
    # Remember that prune_list is sorted by cost.
    pruned_list = []
    for path in path_list: # iterate the ordered path_list from the start to the end ( consecutive so cheaper costs come first )
        last_state = path[-1] # current iterred path last_state
        duplicate_bool = False # the last state doesn t exist in the pruned list
        
        for pruned_path in pruned_list:
            last_pruned_state = pruned_path[-1] # last state for the current iterred pruned path
            
            if eq(last_pruned_state, last_state):
                duplicate_bool = True
                break # if u find duplicated states u just break and tick the bool

        if not duplicate_bool: # because the list is ordered and we should already have the cheapest path in the pruned_list
            pruned_list.append(path)
                
    return pruned_list # Returns a list of pahts

### Ordering
You must order the states in such a way that the first one is the one whose cost plus heuristic is lower. The variables `c` and `h` are, in fact, functions that can be called.

The ordering function determines the algorithm we execute. For example, we could create an `order_depth` that contains:

```python
def order_depth(old_paths, new_paths, *args, **kwargs):
  return new_paths + old_paths
```

#### Tips
- Python has integrated functions such as `sort` and `sorted`. Consult the documentation to see how they work and get the most out of them.
- Use the previous function `prune` to make the final prunning in A*.

In [11]:
# *args and **kwargs are variable arguments; if the argument is not recognised, it is stored in these variables
# Here they are used to ignore unnecessary arguments

def sort_key(path):
    return c(path) + h(path)

def order_astar(old_paths, new_paths, c, h, *args, **kwargs):
    # Sort the list of paths according to heuristics and cost
    new_list = new_paths + old_paths # concatenate the old and new paths
    new_list.sort(key=lambda x: c(x) + h(x[-1])) # sort by cost + heuristic for each state using a lambda function
    return prune(new_list)

### Search Algorithm
Finally, we code the search algorithm in this section. It is about implementing a generic search algorithm.

The inputs will be the functions that we have been developing. First are the specific parts of this particular problem:
- `initial_state`: Initializes a state from the problem
- `expansion`: Expands a state, resulting in all legal states.
- `cost`: Cost function of the problem
- `heuristic`: Heuristic function of the problem
- `solution`: Determines when a problem has reached a solution.
But we can also change the search algorithm we execute
- `ordering`: Ordering function, determines which algorithm we use.

We could declare a new problem by replacing all the previous functions, and if it is correctly programmed, `search` would not have to change in any case. This is what **generic** means, it works for any problem and is independent of the domain.

In [12]:
def search(dataset, initial_state, expansion, cost, heuristic, ordering, solution):
    # Perform a search in the state space
    paths = [[initial_state(dataset)]] # Create the list of paths, initially the initial state

    while paths:
        path = paths.pop(0)

        if solution(path[-1]):
            return path

        expanded_states = expansion(path[-1])
        new_paths = []
        for i in expanded_states:
            new_path = path.copy()
            new_path.append(i)
            new_paths.append(new_path)

        paths = ordering(paths, new_paths, cost, heuristic)
        
    # 1 - While there are paths and no solution has been found
    # 2 - Extract the first path
    # 3 - Check if we are facing a solution state
    # 4.1 - If it is not a solution, expand the path
    # 4.2 - If it is a solution, stop and go to the END step.
    # 5 - For each new expanded state, add it to the path, which generates a list of new paths
    # 6 - Sort the new paths and old paths, and perform pruning. Return to step 1.
    # END - Return the path if it is a solution, otherwise return None

    return None

In [13]:
def measure_sol_search(solution):
  # Measures the cost of the solution found by the search algorithm
  return len(set(solution[-1]["colors"])) # just the color vector as we have no -1 anymore

# Genetic Algorithms
Use the following cells to develop all the code related to genetic algorithms. Remember to respect this general structure and add cells always within each section.

All the code skeleton must be respected to execute the practical assignment correctly. Otherwise, it is possible to make a false step and end up with the wrong conclusions.

## Representation and Fitness Function

### Representation
For the representation, we will use a chromosome with a length equal to the number of vertices (from 0 to N-1). Each vertex $V$ has an index $i$. A gene can take values between 0 and i (inclusive). For example, gene $i=2$ can take colors 0, 1, and 2.

#### Tip
- This representation is decent for the problem, but it is not the best; it can be improved in some cases. Can you think of how?

In [14]:
import random

problem_example = (
                  (0, 1, 1, 0, 1),
                  (1, 0, 1, 0, 0),
                  (1, 1, 0, 1, 1),
                  (0, 0, 1, 0, 0),
                  (1, 0, 1, 0, 0)
                  )

def generate_random_array(length):
    # Generates an array of random elements given the alphabet
    random_array = []
    for i in range(length):
        random_array.append(random.randint(0,i)) # append a random int between 0 and i so we dont have to normalise ( with random() u would have to )
    return random_array

def generate_initial_population(pop_size, *args, **kwargs):
    dataset = kwargs['dataset'] # Dataset with the same structure as the example
    # Obtain the alphabet and length from the dataset
    # Generate an initial population of size pop_size
    initial_pop = []
    length = len(dataset) # length for the chromosome
    for i in range(pop_size):
        initial_pop.append(generate_random_array(length)) # append the generated chromosome
    return initial_pop


### Fitness

On the other hand, the fitness function will be the following. Consider the current number of colors $c$. This is a number we want to minimize, so we will start with:
$$q(x) = \frac{1}{c}$$

Since there are solutions that are not valid, we will penalize the fitness function for each conflict we observe. In particular, we declare $p$ as a count of each edge $A_i$ whose vertices have the same color. For example, if in a connected graph of 3 vertices, we have two colors, there is an edge whose colors are identical, so $p=1$.

The fitness function will be:
$$q'(x) = \frac{1}{c*(p+1)}$$

In [15]:
def fitness(solution, *args, **kwargs):
    dataset = kwargs['dataset']
    # Calculate the profit represented in 'solution'
    # Assume that p is at least 1 to avoid dividing by 0
    
    p = 0
    c = len(set(solution)) # number of unique colors in the solution
    for i in range(len(dataset)): # avoid self counting
        for j in range(i+1, len(dataset)): # avoid counting vertices twice
            if dataset[i][j] == 1 and solution[i] == solution[j]: # if 2 vertices have the same color
                p = p + 1
    q = 1 / (c * (p+1))
    return q

## Genetic Operators

### Roulette Selection
Roulette selection is a selection algorithm where, given a population, it will choose a series of parents. Normally we can use this function in two ways:
1. `number_parents=2` We choose two parents, crossover and mutate; repeating this loop.
2. `number_parents=pop_size` We choose all parents at once, then crossover and mutate iteratively.

In the course, we teach how to do it the first way, but for the code (and results), it doesn't matter how the selection is made.

In [16]:
def roulette_selection(population, fitness, number_parents, *args, **kwargs):
    # Select number_parents individuals from the population using roulette wheel selection
    parents = []
    sum_fitness = sum(fitness)
    if sum_fitness == 0: # uniform random parent fallback (with sum fitness 0 it wont work properly)
        for i in range(number_parents):
            parents.append(random.choice(population).copy())
        return parents
    for i in range(number_parents): # for all parents
        r = random.random() * sum_fitness
        cumulative_fitness = 0
        for parent in range(len(population)):
            cumulative_fitness = cumulative_fitness + fitness[parent] # calculate where the cumulative would be, basically splitting up everyone on a [0,1] axis
            if cumulative_fitness >= r:
                parents.append(population[parent].copy()) # populate the parents array with the copy of selected parent
                break
    return parents

### One-Point Crossover
One-point crossover needs two predecessors. Choosing a random cut point, these predecessors are separated into two parts: a left part and a right part. Child 1 takes the left part of parent 1 and the right part of parent 2.

In [17]:
def one_point_crossover(parent1, parent2, p_cross, *args, **kwargs):
    # Cross two parents with a probability p_cross
    child1, child2 = [], []
    if random.random() < p_cross: # we go in the crossover if positive
        cut_point = random.randint(1, len(parent1)-1) # randomly choose the crossover cut point
        child1 = parent1[0:cut_point] + parent2[cut_point:]
        child2 = parent2[0:cut_point] + parent1[cut_point:] # perform the crossover
    else:
        child1 = parent1.copy()
        child2 = parent2.copy() # identical copies if no crossover
    return child1, child2

### Uniform Mutation
Uniform mutation is checked gene by gene. For each gene, we will check if the probability `p_mut` is met. Each mutation must be a choice from the original alphabet described in the `generate_random_array(...)` function.

In [18]:
def uniform_mutation(chromosome, p_mut, *args, **kwargs):
    dataset = kwargs['dataset'] # Dataset with the same structure as the example
    for i in range(len(chromosome)):
        if random.random() < p_mut:
            chromosome[i] = random.randint(0, i) # if the random probability is viable we assign a random integer to the chromosome
                                                # up to i as in the generate random array function
    return chromosome

### Environmental Selection (Elitism)
Elitism consists of perpetuating the best individual(s) from the previous generation. This is achieved by choosing the best $k$ individuals and inserting them into the new population.

#### Tips:
- Similar to ordering in the search part, reordering the array is also very useful here.

In [19]:
def elitism(population, fitness, offspring, fitness_offspring, *args, **kwargs):
    elite = kwargs['elite_n']
    # Perform generational replacement of the population with elitism.
    # You must return both the new population and its fitness.
    new_pop, new_fit = [], []
    all_pop = population + offspring
    all_fit = fitness + fitness_offspring # combine both fitness and population
    pairs = list(zip(all_pop, all_fit)) # make it into pairs with each element pointing to another in the other all_pop[0] -> all_fit[0]
    pairs.sort(key=lambda x: x[1], reverse=True) # and sort them by the fitness
    for i in range(len(population)): # go up to the max pop_size, so elite_n is covered anyway
        new_pop.append(pairs[i][0])
        new_fit.append(pairs[i][1])
    return new_pop, new_fit

## Genetic Algorithm

## Stopping Condition (Number of Generations)
As soon as the target generation is reached, the algorithm stops and the population of solutions will be the result.

In [20]:
def generation_stop(generation, fitness, *args, **kwargs):
    max_gen=kwargs['max_gen']
    # Check whether the stopping criterion (maximum number of generations) is met.
    if generation >= max_gen:
        return True
    return False

### Generic Algorithm
There are several hyperparameters to adjust for executing the genetic algorithm. Use these default parameters, but don't forget to explore each one later.

Parameter | Variable | Value
--- | --- | ---
Population size | `pop_size` | 100
Offspring size | `offspring_size` | 100
Crossover probability | `p_cross` | 0.8
Mutation probability | `p_mut` | 0.1
Maximum generation | `max_gen` | 100
Number of elites | `elite_n` | 2

For example, it is recommended later to change `p_mut` to $1/||V||$ (1 divided by the number of vertices). It is necessary to explore these parameters for the practice and observe how it affects the fitness (as well as the result).

In [21]:
def genetic_algorithm(generate_population, pop_size, fitness_function, stopping_criteria, offspring_size,
                      selection, crossover, p_cross, mutation, p_mut, environmental_selection, *args, **kwargs):
    # Apply a genetic algorithm to a maximisation problem
    population = generate_population(pop_size, *args, **kwargs) # Creates a population of individuals of size pop_size
    fitness = [fitness_function(item, *args, **kwargs) for item in population] # Contains the evaluation of the population
    best_fitness = [] # Stores the best fitness of each generation
    mean_fitness = [] # Stores the average fitness of each generation
    generation = 0 # Generation counter

    # 1 - Initialise the population with the generate_population function
    # 2 - Evaluate the population with the fitness_function function
    # 3 - While the stopping_criteria criterion is not met
    # 4 - Select parents with the selection function
    # 5 - Cross parents using the crossover function with probability p_cross
    # 6 - Mutation of offspring using the mutation function with probability p_mut
    # 7 - Evaluation of offspring
    # 8 - Generation of the new population using the environmental_selection function

    
    # the following code utilises the exact order which is told above
    
    while not stopping_criteria(generation, fitness, *args, **kwargs):
        parents = selection(population, fitness, pop_size, *args, **kwargs) # select parents
        if len(parents) % 2 == 1:
            parents.append(parents[-1].copy()) # if it is a odd number, we should change that, by adding one more so we have pairs
        new_pop = [] # list to collect offspring
        for i in range(1, len(parents), 2): # iterate over parents in pairs and do crossover + mutation for each pair
            child1, child2 = crossover(parents[i-1], parents[i], p_cross, *args, **kwargs)
            child1 = mutation(child1, p_mut, *args, **kwargs)
            child2 = mutation(child2, p_mut, *args, **kwargs)
            new_pop.append(child1)
            new_pop.append(child2) # append offspring
        new_fit = [] # calculate the fitness for the new population
        for solution in new_pop:
            new_fit.append(fitness_function(solution, *args, **kwargs))
        population, fitness = environmental_selection(population, fitness, new_pop, new_fit, *args, **kwargs) # returns the new gen
        best_fitness.append(max(fitness))
        mean_fitness.append(sum(fitness) / len(fitness))
        generation += 1
    return population, fitness, generation, best_fitness, mean_fitness

In [22]:
def measure_sol_genetic(output):
  # output is the output of genetic_algorithm. That is := (population, fitness, generation, best_fitness, mean_fitness)
  # Measures the BEST fitness found by genetic algorithm
  # get index from fitness
    benefit = None
    population, fitness, generation, best_fitness, mean_fitness = output
    if not population or not fitness:
        return None
    return len(set(population[fitness.index(max(fitness))])) # number of unique colors used by population[index of the most fittest] 

# General
Use the `search_run` and `genetic_run` functions to extract results.

## Utilities
Use these pre-programmed functions to run the experiments and summarize the code.

- `search_run` receives all the functions you previously programmed in a totally fixed manner. You only need to explore the heuristic.
- `genetic_run` receives the previously programmed functions dynamically. Explore each parameter freely.

### Timer

In [23]:
############################### DO NOT TOUCH ###############################
#                                                                          #
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print("Execution time: ", end - start, " seconds")
        return res
    return wrapper
#                                                                          #
############################### DO NOT TOUCH ###############################

# This code times the execution of any function

### Wrappers

In [24]:
############################### DO NOT TOUCH ###############################
#                                                                          #
@timer
def search_run(dataset, heuristic):
    output = search(dataset, initial_state, expand, cost, heuristic, order_astar, is_solution)
    print(f"Graph: {output[-1]}")
    return measure_sol_search(output)

@timer
def genetic_run(generate_population, pop_size, fitness_function, stopping_criteria,
                offspring_size, selection, crossover, p_cross, mutation, p_mut,
                environmental_selection, dataset, *args, **kwargs):
    output =  genetic_algorithm(generate_population,
                                pop_size,
                                fitness_function,
                                stopping_criteria,
                                offspring_size,
                                selection,
                                crossover,
                                p_cross,
                                mutation,
                                p_mut,
                                environmental_selection,
                                dataset=dataset,
                                *args, **kwargs)
    return measure_sol_genetic(output)
#                                                                          #
############################### DO NOT TOUCH ###############################


### Problems

Consider that if a problem takes more than 5 minutes to solve, the algorithm will take too long to be solved.

In [25]:
# 5-vertex graph:
problem_5=(
(0, 1, 1, 1, 1),
(1, 0, 1, 1, 1),
(1, 1, 0, 1, 1),
(1, 1, 1, 0, 1),
(1, 1, 1, 1, 0),
)

# 8-vertex graph:
problem_8=(
(0, 1, 1, 1, 1, 1, 0, 1),
(1, 0, 1, 1, 1, 0, 1, 1),
(1, 1, 0, 1, 0, 1, 1, 0),
(1, 1, 1, 0, 1, 0, 0, 1),
(1, 1, 0, 1, 0, 1, 1, 1),
(1, 0, 1, 0, 1, 0, 1, 1),
(0, 1, 1, 0, 1, 1, 0, 1),
(1, 1, 0, 1, 1, 1, 1, 0),
)

# 12-vertex graph:
problem_12=(
(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1),
(1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1),
(1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1),
(1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0),
(1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1),
(1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0),
(1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0),
(1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0),
(0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1),
(1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0),
(1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1),
(1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0),
)

# 18-vertex graph:
problem_18=(
(0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1),
(1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1),
(0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0),
(0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0),
(0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1),
(1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0),
(0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0),
(1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0),
(0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0),
(0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0),
(1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1),
(0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1),
(0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1),
(1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0),
(1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1),
(0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0),
(0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1),
(1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0),
)

# 25-vertex graph:
problem_25=(
(0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1),
(1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0),
(0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0),
(0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0),
(0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0),
(0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0),
(1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0),
(0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0),
(0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0),
(0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1),
(0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0),
(1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1),
(0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0),
(0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0),
(0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1),
(0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0),
(1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0),
(0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0),
(0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1),
(1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0),
(0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0),
(0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0),
(0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0),
(0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1),
(1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0),
)
problems = [problem_5, problem_8, problem_12, problem_18, problem_25]

## Executions
This space in the practice is reserved for the execution of the algorithms. It is recommended to use the launch_experiment method.

In [27]:
#@title A* execution
for i, problem in enumerate(problems):
    
    print(f"Heuristic h=0")
    print(f"Problem: {i}")
    search_benefit = search_run(problem, heuristic=lambda s: 0)
    print(f"Search: {search_benefit}\n")
    
    print(f"Heuristic h=len(state['colors']) - state['nextnode']")
    print(f"Problem: {i}")
    h_new = lambda state: len(state["colors"]) - state["nextnode"]
    search_benefit = search_run(problem, heuristic=h_new)
    print(f"Search: {search_benefit}\n")
    
    print(f"Heuristic h=heuristic (the active heuristic)")
    print(f"Problem: {i}")
    search_benefit = search_run(problem, heuristic=heuristic)
    print(f"Search: {search_benefit}\n")
    
    print("-----\n\n")


Heuristic h=0
Problem: 0
Graph: {'graph': ((0, 1, 1, 1, 1), (1, 0, 1, 1, 1), (1, 1, 0, 1, 1), (1, 1, 1, 0, 1), (1, 1, 1, 1, 0)), 'colors': [0, 1, 2, 3, 4], 'nextnode': 5}
Execution time:  4.1484832763671875e-05  seconds
Search: 5

Heuristic h=len(state['colors']) - state['nextnode']
Problem: 0
Graph: {'graph': ((0, 1, 1, 1, 1), (1, 0, 1, 1, 1), (1, 1, 0, 1, 1), (1, 1, 1, 0, 1), (1, 1, 1, 1, 0)), 'colors': [0, 1, 2, 3, 4], 'nextnode': 5}
Execution time:  2.4080276489257812e-05  seconds
Search: 5

Heuristic h=heuristic (the active heuristic)
Problem: 0
Graph: {'graph': ((0, 1, 1, 1, 1), (1, 0, 1, 1, 1), (1, 1, 0, 1, 1), (1, 1, 1, 0, 1), (1, 1, 1, 1, 0)), 'colors': [0, 1, 2, 3, 4], 'nextnode': 5}
Execution time:  4.8160552978515625e-05  seconds
Search: 5

-----


Heuristic h=0
Problem: 1
Graph: {'graph': ((0, 1, 1, 1, 1, 1, 0, 1), (1, 0, 1, 1, 1, 0, 1, 1), (1, 1, 0, 1, 0, 1, 1, 0), (1, 1, 1, 0, 1, 0, 0, 1), (1, 1, 0, 1, 0, 1, 1, 1), (1, 0, 1, 0, 1, 0, 1, 1), (0, 1, 1, 0, 1, 1, 0, 1), (1, 

## A* result analysis

I implemented A* algorithm using the state representation, expansion, cost and heuristic functions. I ran three heuristic variants, represented below. For each heuristic I ran A* on all problems:
### 1. h = 0
- Admissible trivial heuristic, but uninformed, so it is really slow but it returns the most optimal cost.
  | Problem | Algorithm | Time | Benefit |
|----------|-----------|--------|-----------|
| P0       |  A*       | 0.000041s     | 5         |
| P1       |  A*      | 0.000087s     | 5         |
| P2       |  A*       | 0.0016s    | 5         |
| P3       |  A*       | 0.45s     | 5         |
| P4       |  A*      | 208.60s     | 5         |
### 2. h = len(state["colors"]) - state["nextnode"]) (estimates the number of remaining nodes)
- Very fast and cheap heuristic ( O(1) ), the fastest out of these 3, but unadmissible and not so well informed, getting bad quality results in the more complicated datasets.
  | Problem | Algorithm | Time | Benefit |
|----------|-----------|--------|-----------|
| P0       |  A*       | 0.000024s     | 5         |
| P1       |  A*      | 0.0023s     | 5         |
| P2       |  A*       | 0.00036s    | 5         |
| P3       |  A*       | 0.00029s     | 6         |
| P4       |  A*      | 0.0010s     | 7         |
### 3. h = active heuristic which verifies for each remaining uncolored node if it s necessary a new color (u can see the code in the heuristic function above)
- More costy, but really well informed, I found this unadmissible heuristic the best as it keeps a balance between speed and the quality of the solution, getting the same results as the admissible one but much much faster. (I explained about admissibility in the active function if u scroll up again)
  | Problem | Algorithm | Time | Benefit |
|----------|-----------|--------|-----------|
| P0       |  A*       | 0.000048s     | 5         |
| P1       |  A*      | 0.00044s     | 5         |
| P2       |  A*       | 0.0014s    | 5         |
| P3       |  A*       | 0.0052s     | 5         |
| P4       |  A*      | 0.16s     | 5         |
### Conclusion
- So, using the first heuristic as reference of corectitude, I concluded the third heuristic, and the one that is implemented in the active heuristic function is the best, getting the correct results the fastest.

In [26]:
for j in range(3):
    print(f"Run {j}\n-----\n")
    for i, problem in enumerate(problems):
        print(f"Config 1")
        print(f"Problem: {i}")
        ga_benefit = genetic_run(generate_population=generate_initial_population, pop_size=50,
                                     fitness_function=fitness,
                                     stopping_criteria=generation_stop,
                                     offspring_size=50,
                                     selection=roulette_selection,
                                     crossover=one_point_crossover, p_cross=0.9,
                                     mutation=uniform_mutation, p_mut= 1/len(problem),
                                     environmental_selection= elitism,
                                     dataset=problem,
                                     max_gen=100,
                                    elite_n=2)
        print(f"Genetic: {ga_benefit}\n")
    print("\n-----\n")
        
    for i, problem in enumerate(problems):
        print(f"Config 2")
        print(f"Problem: {i}")
        ga_benefit = genetic_run(generate_population=generate_initial_population, pop_size=100,
                                     fitness_function=fitness,
                                     stopping_criteria=generation_stop,
                                     offspring_size=100,
                                     selection=roulette_selection,
                                     crossover=one_point_crossover, p_cross=0.9,
                                     mutation=uniform_mutation, p_mut= 2/len(problem),
                                     environmental_selection= elitism,
                                     dataset=problem,
                                     max_gen=100,
                                    elite_n=5)
        print(f"Genetic: {ga_benefit}\n")
    print("\n-----\n")
    
    for i, problem in enumerate(problems):
        print(f"Config 3")
        print(f"Problem: {i}")
        ga_benefit = genetic_run(generate_population=generate_initial_population, pop_size=200,
                                     fitness_function=fitness,
                                     stopping_criteria=generation_stop,
                                     offspring_size=200,
                                     selection=roulette_selection,
                                     crossover=one_point_crossover, p_cross=0.9,
                                     mutation=uniform_mutation, p_mut= 1/len(problem),
                                     environmental_selection= elitism,
                                     dataset=problem,
                                     max_gen=150,
                                    elite_n=5)
        print(f"Genetic: {ga_benefit}")
    print("\n-----\n")

Run 0
-----

Config 1
Problem: 0
Execution time:  0.024201631546020508  seconds
Genetic: 5

Config 1
Problem: 1
Execution time:  0.03032541275024414  seconds
Genetic: 5

Config 1
Problem: 2
Execution time:  0.07181906700134277  seconds
Genetic: 5

Config 1
Problem: 3
Execution time:  0.06787395477294922  seconds
Genetic: 6

Config 1
Problem: 4
Execution time:  0.09228134155273438  seconds
Genetic: 8


-----

Config 2
Problem: 0
Execution time:  0.06941819190979004  seconds
Genetic: 5

Config 2
Problem: 1
Execution time:  0.08730077743530273  seconds
Genetic: 5

Config 2
Problem: 2
Execution time:  0.08091044425964355  seconds
Genetic: 5

Config 2
Problem: 3
Execution time:  0.1704397201538086  seconds
Genetic: 6

Config 2
Problem: 4
Execution time:  0.22203683853149414  seconds
Genetic: 7


-----

Config 3
Problem: 0
Execution time:  0.26343417167663574  seconds
Genetic: 5
Config 3
Problem: 1
Execution time:  0.2897682189941406  seconds
Genetic: 5
Config 3
Problem: 2
Execution time:  0

## Genetic Algorithm Analysis

I implemented the algorithm with the chromosome representation, fitness, roulette selection, one‑point crossover and uniform mutation.
I tested several parameter configs, modifying: population size, mutation rate and elitism. 

For each configuration, I ran the algorithm three times in order to account for the stochastic nature of genetic algorithms and reported both execution time and solution benefit (number of colors used).

### Config 1

- Config: pop_size=50, offspring_size=50, p_cross=0.9, p_mut= 1/len(problem) ,max_gen=100, elite_n=2

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| Run 0   |               |          |         |
| P0      | GA (Config 1) | 0.024    | 5       |
| P1      | GA (Config 1) | 0.030    | 5       |
| P2      | GA (Config 1) | 0.072    | 5       |
| P3      | GA (Config 1) | 0.068    | 6       |
| P4      | GA (Config 1) | 0.092    | 8       |
| Run 1   |               |          |         |
| P0      | GA (Config 1) | 0.021    | 5       |
| P1      | GA (Config 1) | 0.027    | 5       |
| P2      | GA (Config 1) | 0.043    | 6       |
| P3      | GA (Config 1) | 0.061    | 7       |
| P4      | GA (Config 1) | 0.108    | 9       |
| Run 2   |               |          |         |
| P0      | GA (Config 1) | 0.034    | 5       |
| P1      | GA (Config 1) | 0.048    | 5       |
| P2      | GA (Config 1) | 0.047    | 5       |
| P3      | GA (Config 1) | 0.069    | 7       |
| P4      | GA (Config 1) | 0.106    | 8       |


#### Averages

- This config prioritizes speed over exploration. With a smaller pop_size=50 and minimal elitism (2 individuals), it runs very fast on all problems. It performs the best on smaller problems (P0,P1,P2), consistently finding 5-color solutions. However, on more complex graphs (P3,P4), the limited diversity becomes a constraint: P3 often requires 6–7 colors, while P4 occasionally reaches 9 colors.

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| P0      | GA (Config 1) | 0.026    | 5.00    |
| P1      | GA (Config 1) | 0.035    | 5.00    |
| P2      | GA (Config 1) | 0.054    | 5.33    |
| P3      | GA (Config 1) | 0.066    | 6.67    |
| P4      | GA (Config 1) | 0.102    | 8.33    |




### Config 2

- Config: pop_size=100, offspring_size=100, p_cross=0.9, p_mut=2/len(problem), max_gen=100, elite_n=5

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| Run 0   |               |          |         |
| P0      | GA (Config 2) | 0.069    | 5       |
| P1      | GA (Config 2) | 0.087    | 5       |
| P2      | GA (Config 2) | 0.081    | 5       |
| P3      | GA (Config 2) | 0.170    | 6       |
| P4      | GA (Config 2) | 0.222    | 7       |
| Run 1   |               |          |         |
| P0      | GA (Config 2) | 0.060    | 5       |
| P1      | GA (Config 2) | 0.094    | 5       |
| P2      | GA (Config 2) | 0.140    | 5       |
| P3      | GA (Config 2) | 0.166    | 6       |
| P4      | GA (Config 2) | 0.261    | 8       |
| Run 2   |               |          |         |
| P0      | GA (Config 2) | 0.065    | 5       |
| P1      | GA (Config 2) | 0.083    | 5       |
| P2      | GA (Config 2) | 0.115    | 5       |
| P3      | GA (Config 2) | 0.132    | 6       |
| P4      | GA (Config 2) | 0.249    | 7       |


#### Averages

- This config increases mutation pressure while keeping a moderate population size. The higher mutation rate improves exploration stability but does not significantly improve solution quality for smaller problems. P0–P2 consistently find 5 colors (which is the optimal path), while P3 stabilizes at 6 colors. On the hardest problem (P4), results vary between 7 and 8 colors, indicating improved exploration compared to Config 1 but still some variability.

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| P0      | GA (Config 2) | 0.065    | 5.00    |
| P1      | GA (Config 2) | 0.088    | 5.00    |
| P2      | GA (Config 2) | 0.112    | 5.00    |
| P3      | GA (Config 2) | 0.156    | 6.00    |
| P4      | GA (Config 2) | 0.244    | 7.33    |







### Config 3

- Config: pop_size=200, offspring_size=200, p_cross=0.9, p_mut=2/len(problem), max_gen=150, elite_n=5

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| Run 0   |               |          |         |
| P0      | GA (Config 3) | 0.263    | 5       |
| P1      | GA (Config 3) | 0.290    | 5       |
| P2      | GA (Config 3) | 0.355    | 5       |
| P3      | GA (Config 3) | 0.500    | 5       |
| P4      | GA (Config 3) | 0.743    | 7       |
| Run 1   |               |          |         |
| P0      | GA (Config 3) | 0.275    | 5       |
| P1      | GA (Config 3) | 0.325    | 5       |
| P2      | GA (Config 3) | 0.366    | 5       |
| P3      | GA (Config 3) | 0.555    | 6       |
| P4      | GA (Config 3) | 0.782    | 7       |
| Run 2   |               |          |         |
| P0      | GA (Config 3) | 0.226    | 5       |
| P1      | GA (Config 3) | 0.309    | 5       |
| P2      | GA (Config 3) | 0.357    | 5       |
| P3      | GA (Config 3) | 0.564    | 6       |
| P4      | GA (Config 3) | 0.745    | 7       |


#### Averages

- This config prioritizes exploration and solution stability over execution speed. With a large population size (pop_size=200), extended evolution (max_gen=150), and strong elitism (5 individuals), it significantly increases computational cost. For simpler problems (P0, P1, P2), the algorithm consistently finds the optimal 5-color solutions. On more complex problems (P3, P4), the larger population helps stabilize results: P3 mostly converges to 5–6 colors, while P4 consistently finds 7-color solutions.

| Problem | Algorithm     | Time (s) | Benefit |
| ------- | ------------- | -------- | ------- |
| P0      | GA (Config 3) | 0.255    | 5.00    |
| P1      | GA (Config 3) | 0.308    | 5.00    |
| P2      | GA (Config 3) | 0.359    | 5.00    |
| P3      | GA (Config 3) | 0.540    | 5.67    |
| P4      | GA (Config 3) | 0.757    | 7.00    |

- Overall, this configuration achieves the most stable benefits across runs, but at the cost of substantially longer execution times (though less than a second even for the hardest problem).

## Final Conclusion

- Both A* and Genetic Algorithm successfully solve the graph coloring problem, but they have advantages and disadvantages.

- A* guarantees optimal solutions when using admissible heuristics, but the execution time varies drastically depending on the problem and heuristic. For large or complex graphs (P4), even a well-informed heuristic can take significantly longer, while for smaller problems, it is extremely fast.

- The Genetic Algorithm provides a stochastic approach, switching strict optimality with flexibility. By tuning/modifying population size, mutation rate, and elitism, the algorithm can quickly find good solutions across all problems, with more consistent performance for harder graphs (P3, P4). However, it requires careful configuration to balance exploration and speed.

| Algorithm                 | Average Time (s) | Average Benefit
| ------------------------- | ------------- | ------------
| A* (admissible heuristic=0) | 69.80         | 5.00        
| A* (active heuristic)     | 0.032         | 5.00      
| GA Config 1               | 0.057         | 6.67   
| GA Config 2               | 0.156         | 6.00      
| GA Config 3               | 0.757         | 5.67     

- A* is ideal for guaranteed optimal solutions on small graphs, while GA excels at larger, more complex instances where near-optimal solutions are acceptable and runtime needs to remain reasonable. The active heuristic in A* and the larger population configurations in GA represent the best trade-offs in terms of speed and solution quality.
- Even though here the GA is still slower than A* with the active heuristic, I am sure that for much larger datasets/graphs the GA would produce better results, more close to the optimal path and in a good time.